In [7]:
import os
import datetime
import time
import requests
import pandas as pd
from dotenv import load_dotenv

In [4]:
load_dotenv(".env", override=True)
POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")

In [8]:
def fetch_polyio_crypto_price(crypto_ticker, multiplier, timespan, from_date, to_date, to_adjust):
    from_date = datetime.datetime.strptime(from_date, "%Y-%m-%d")
    to_date = datetime.datetime.strptime(to_date, "%Y-%m-%d")
    all_data = []

    current_date = from_date
    start_time = time.time()
    while current_date <= to_date:
        next_date = current_date + datetime.timedelta(days=30)
        if next_date > to_date:
            next_date = to_date

        start_str = current_date.strftime("%Y-%m-%d")
        end_str = next_date.strftime("%Y-%m-%d")

        url = (f"https://api.polygon.io/v2/aggs/ticker/{crypto_ticker}/"
               f"range/{multiplier}/{timespan}/{start_str}/{end_str}?"
               f"adjusted={to_adjust}&sort=asc&limit=50000&apiKey={POLYGON_API_KEY}")
        
        try:
            start_time = time.time()
            response = requests.get(url)
            
            if response.status_code != 200:
                raise Exception(f"API request failed with status code {response.status_code}")

            data = response.json()
            if "results" not in data or not data["results"]:
                print(f"No results found for {start_str} to {end_str}")
            else:
                df = pd.DataFrame(data["results"])
                df["date-time"] = df["t"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000, datetime.UTC))
                df = df.rename(columns={"c": "close_price", "h": "high_price", "l": "low_price", "o": "open_price", "v": "volume"})
                df = df[["date-time", "open_price", "high_price", "low_price", "close_price", "volume"]]
                all_data.append(df)

        except Exception as e:
            print(f"Error fetching data from {start_str} to {end_str}: {e}")
        
        elapsed_time = time.time() - start_time
        remaining_time = max(15 - elapsed_time, 0)
        if remaining_time > 0:
            time.sleep(remaining_time)
        current_date = next_date + datetime.timedelta(days=1)
        
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame()

In [9]:
tickers = ["X:ETHUSD", "X:BTCUSD", "X:USDCUSD", "X:USDTUSD"]
split_adjusted_values = ["true", "false"]

In [ ]:
for ticker in tickers:
    for split_adj in split_adjusted_values:
        file_path = ("./polyio_data/" + 
                     f"{'adjusted' if split_adj == 'true' else 'unadjusted'}/" +
                     f"{ticker.split(':')[1][:-3]}.csv")
        
        if os.path.exists(file_path):
            user_input = input(f"{file_path} already exists! Do you want to fetch again and replace the file? (y/n): ").strip().lower()
            
            if user_input == 'y':
                res = fetch_polyio_crypto_price(ticker, 1, "hour", "2023-03-01", "2025-03-08", split_adj)
                res.to_csv(file_path, index=True)
                print(f"{file_path} has been replaced and saved.")
            else:
                print(f"Skipping {file_path}. No changes made.")
        else:
            res = fetch_polyio_crypto_price(ticker, 1, "hour", "2023-03-01", "2025-03-08", split_adj)
            res.to_csv(file_path, index=True)
            print(f"{file_path} saved.")